# Exploration of rating


In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import datetime 

import sqlite3
import sys
import time
import tqdm
from tqdm.auto import tqdm
import pickle
import joblib
import os

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'


## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [146]:
# # Function to get the table names
# def get_table_names(conn):
#     query = "SELECT name FROM sqlite_master WHERE type='table';"
#     return pd.read_sql(query, conn)['name'].tolist()

# # Function to load tables into DataFrames
# def load_tables_to_dfs(conn):
#     table_names = get_table_names(conn)
#     dataframes = {}
    
#     for table in table_names:
#         # Load table into a DataFrame
#         df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
#         # Detect and convert JSON formatted columns (if any)
#         for col in df.columns:
#             # Check if any entry in the column is a valid JSON (list or dictionary)
#             if df[col].apply(lambda x: isinstance(x, str)).all():
#                 try:
#                     # Try parsing the column as JSON
#                     df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
#                 except (json.JSONDecodeError, TypeError):
#                     # If it fails, skip the column
#                     pass
        
#         # Store the DataFrame with table name + '_df'
#         dataframes[f"{table}_df"] = df
        
#     return dataframes

# if os.path.exists(data_path + 'dfs_dict.pkl'):
#     cell_has_run = True
#     # Load the dictionary of DataFrames from the pickle
#     with open(data_path + 'dfs_dict.pkl', 'rb') as f:
#         dfs = pickle.load(f)
# # Check if the flag variable exists in the global scope so that this code does not run twice
# if 'cell_has_run' not in globals():
#     path = + data_path + "melee_player_database.db"
    
#     # Connect to the database
#     conn = sqlite3.connect(path)

#     # Convert each table into a DataFrame
#     dfs = load_tables_to_dfs(conn)

#     # Close the connection
#     conn.close()

#     # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
#     # For example, to access the DataFrame for a table called 'players':
#     # players_df = dfs['players_df']

#     dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
#     dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
#     # Set the flag to indicate that the cell has been run
#     cell_has_run = True

### Load Data


In [147]:
players_df = pd.read_pickle(data_path + '/labelled_data/players_df.pkl')
players_df.head()


,game,player_id,tag,all_tags,prefixes,social,country,state,region,c_country,c_state,c_region,placings,characters,alias
0,melee,Rishi,Rishi,[Rishi],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
1,melee,15634,lloD,"[lloD, VGz | lloD, Llod]",[],{'twitter': ['lloD74']},United States,VA,,US,CA,Laurel,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/peach': 1089, 'melee/falco': 1, 'melee...",None
2,melee,6126,Zain,"[Zain, DontTestMe]",[PG],{'twitter': ['PG_Zain']},United States,VA,,US,CA,Los Angeles,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/marth': 1065, 'melee/pichu': 1, 'melee...",DontTestMe
3,melee,Chu,Chu,[Chu],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
4,melee,5620,Junebug,"[Junebug, LS | VGz Junebug]",[],{'twitter': ['arJunebug']},United States,VA,,US,VA,Richmond,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/sheik': 46, 'melee/falco': 4, 'melee/g...",None


In [148]:
sets_df = pd.read_pickle(data_path + '/labelled_data/sets_df.pkl')
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data")
print(sets_df.shape)
sets_df.head()


32.9% percent of sets have some game data
(1795681, 19)


,key,game,tournament_key,winner_id,p1_id,p2_id,p1_score,p2_score,location_names,bracket_name,bracket_order,set_order,best_of,game_data,top_8,top_8_location_names,valid_top_8_bracket,top_8_bracket_location_names,major
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,5620,Chillin,3,1,"[R1, Round 1, Round 1]",,1,A,5,[],False,,False,,False
1,104675844,melee,mdva-invitational-2017-(challonge-mirror),Aglet,15634,Aglet,2,3,"[R1, Round 1, Round 1]",,1,B,5,[],False,,False,,False
2,104675845,melee,mdva-invitational-2017-(challonge-mirror),6126,6126,1097,3,0,"[R1, Round 1, Round 1]",,1,C,5,[],False,,False,,False
3,104675846,melee,mdva-invitational-2017-(challonge-mirror),1069,Chu,1069,0,3,"[R1, Round 1, Round 1]",,1,D,5,[],False,,False,,False
4,104675847,melee,mdva-invitational-2017-(challonge-mirror),Rishi,Jerry,Rishi,1,3,"[R1, Round 1, Round 1]",,1,E,5,[],False,,False,,False


In [149]:
tournament_info_df = pd.read_pickle(data_path + '/labelled_data/tournament_info_df.pkl')
print(tournament_info_df.shape)
tournament_info_df.head()


(39675, 37)


,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,...,WSF_B_p2,LN_A_p1_non_top_8_sets,LN_A_p2_non_top_8_sets,LN_B_p1_non_top_8_sets,LN_B_p2_non_top_8_sets,WSF_A_p1_non_top_8_sets,WSF_A_p2_non_top_8_sets,WSF_B_p1_non_top_8_sets,WSF_B_p2_non_top_8_sets,major
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,...,Ginger,"[(32, True), (62, True), (77, False), (164, Tr...","[(39, True), (65, True), (78, False), (165, Tr...","[(47, True), (69, True), (80, False), (159, Tr...","[(40, True), (66, True), (79, True), (85, Fals...","[(28, True), (60, True), (76, True), (84, True)]","[(36, True), (64, True), (78, True), (85, True)]","[(44, True), (68, True), (80, True), (86, True)]","[(52, True), (72, True), (82, True), (87, True)]",NaN
2,melee,slippi-champions-league-week-1__melee-singles,Slippi Champions League Week 1,pgstats,slippi-champions-league-week-1,melee-singles,20,,2020-10-11 14:00:00,2020-10-11 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,melee,slippi-champions-league-week-2__melee-singles,Slippi Champions League Week 2,pgstats,slippi-champions-league-week-2,melee-singles,20,,2020-10-18 14:00:00,2020-10-18 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,melee,slippi-champions-league-week-3__melee-singles,Slippi Champions League Week 3,pgstats,slippi-champions-league-week-3,melee-singles,20,,2020-10-25 14:00:00,2020-10-25 14:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [150]:
overall_players_ranking_new_weekly_df = pd.read_pickle(data_path + 'overall_players_ranking_new_weekly.pkl')
overall_players_ranking_new_weekly_df.sample(3)

,1617201,1497667,2653190,3657740,41449,6039,3621289,3682293,3332271,2034855,...,1911774,4106746,Black hayato,The boy,138692,3293641,1701726,2408932,SmyD,15495
2022-09-15,1305.337441,1500.0,1500.0,1500.0,1517.697778,1362.829389,1500.0,1500.0,1500.0,1500.0,...,1292.611023,1500.0,1337.689105,1400.124736,1397.772865,1500.0,1500.0,1514.677506,1559.078031,1482.878104
2019-05-09,1500.000000,1500.0,1500.0,1500.0,1147.194923,1362.829389,1500.0,1500.0,1500.0,1500.0,...,1500.000000,1500.0,1337.689105,1400.124736,1397.772865,1500.0,1500.0,1500.000000,1559.078031,1482.878104
2019-05-23,1500.000000,1500.0,1500.0,1500.0,1147.194923,1362.829389,1500.0,1500.0,1500.0,1500.0,...,1500.000000,1500.0,1337.689105,1400.124736,1397.772865,1500.0,1500.0,1500.000000,1559.078031,1482.878104


In [151]:
overall_players_rds_new_weekly_df = pd.read_pickle(data_path + 'overall_players_rds_new_weekly.pkl')
overall_players_rds_new_weekly_df.sample(3)

,1617201,1497667,2653190,3657740,41449,6039,3621289,3682293,3332271,2034855,...,1911774,4106746,Black hayato,The boy,138692,3293641,1701726,2408932,SmyD,15495
2024-05-02,281.242066,274.899939,199.37809,253.228616,44.848049,305.731548,235.474376,177.365549,245.771455,181.275559,...,101.666165,350.0,350.000000,318.772147,298.413216,129.236864,245.223594,225.017359,282.838002,255.443119
2018-08-09,350.000000,350.000000,350.00000,350.000000,267.431168,246.962116,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.0,318.683097,262.936281,237.845842,350.000000,350.000000,350.000000,217.993925,181.042598
2016-12-08,350.000000,350.000000,350.00000,350.000000,249.127495,227.022356,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.0,303.491690,244.304582,350.000000,350.000000,350.000000,350.000000,195.120008,191.244899


In [152]:
char_vs_char_player_rankings_weekly_alt2_df = pd.read_pickle(data_path + 'char_vs_char_player_rankings_weekly_alt2.pkl')
char_vs_char_player_rankings_weekly_alt2_df.sample(3)


,3688504/sheik/roy,3688504/sheik/samus,3688504/sheik/sheik,3689802/sheik/marth,3689821/falco/falco,3689821/falco/fox,3689821/falco/jigglypuff,3689821/falco/kirby,3689821/falco/marth,3689821/falco/mewtwo,...,368847/fox/pikachu,368847/fox/samus,368847/fox/sheik,3688504/sheik/captainfalcon,3688504/sheik/falco,3688504/sheik/fox,3688504/sheik/jigglypuff,3688504/sheik/luigi,3688504/sheik/marth,3688504/sheik/peach
2016-08-25,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,...,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
2015-11-26,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,...,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
2017-07-13,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,...,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0


In [153]:
char_vs_char_player_rankings_weekly_alt2_rds_df = pd.read_pickle(data_path + 'char_vs_char_player_rankings_weekly_alt2_rds.pkl')
char_vs_char_player_rankings_weekly_alt2_rds_df.sample(3)

,1960158/fox/mewtwo,1960158/fox/roy,1960158/jigglypuff/falco,1960158/jigglypuff/fox,1960158/jigglypuff/iceclimbers,1960158/jigglypuff/jigglypuff,1960158/jigglypuff/link,1960158/jigglypuff/luigi,1960158/jigglypuff/marth,1960158/jigglypuff/sheik,...,999886/luigi/falco,999886/marth/captainfalcon,999886/sheik/falco,999888/sheik/captainfalcon,999888/sheik/falco,999888/sheik/fox,999888/sheik/jigglypuff,999888/sheik/marth,999888/sheik/mrgameandwatch,999888/sheik/samus
2023-06-22,350.000000,350.000000,156.656845,112.795715,310.666908,164.443799,309.166132,258.784427,189.739970,185.036743,...,273.827340,249.142992,287.838956,195.906353,221.061213,225.680665,247.177894,173.811315,255.189216,255.189216
2017-07-27,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
2024-01-11,240.240445,210.633639,166.402706,123.897833,315.698343,173.759944,314.220094,264.801280,197.866144,193.361567,...,279.520995,255.386946,293.261481,203.812215,228.086324,232.556903,253.470243,182.651554,261.291132,261.291132


In [154]:
top_player_id = {
    'aMSa': '1021',
    'Cody': '19554',
    'Mang0': '1000',
    'Zain': '6126',
    'Armada': '6189',
    'Wizzrobe': '1028',
    'Mew2King': '1003',
    'PPMD': '1002',
    'Hungrybox': '1004',
    'Plup': '15990',
    'Axe': '16342'
}

# players_df[players_df['tag']=='Hungrybox']
# players_df.loc[100:120]



I want a function that quickly finds a player's alt_2 and alt_2_rds for all relevant matchups.

- Identify columns

In [155]:
# mango = '1000'

# char_char_columns = char_vs_char_player_rankings_weekly_alt2_df.columns
# print(char_char_columns[:10])

# char_char_cols_np = np.array(char_char_columns)
# char_rating_cols_argsort = np.argsort(char_char_cols_np)
# char_char_cols_sort = np.sort(char_char_cols_np)


# char_char_player_ids = [name.split('/')[0] for name in char_char_columns]
# char_char_player_ids = np.array(char_char_player_ids)
# char_char_player_ids_argsort = np.argsort(char_char_player_ids)

Find all mango's columns

In [156]:

# def find_player_columns(player_id):
#     '''
#     Finds all indices in `char_char_columns` where the entry starts with `player_id`.
    
#     Args:
#     - player_id (str): The player ID to search for.
    
#     Returns:
#     - indices (np.ndarray): Array of indices in `char_char_columns` where the entry starts with `player_id`.
#     '''
    
#     # First, get a sorted version of `char_char_columns` and the indices for sorting
#     sorted_indices = np.argsort(char_char_player_ids)
#     sorted_char_char_player_ids = char_char_player_ids[sorted_indices]
    
#     # Use `np.searchsorted` to find the range of indices where `player_id` occurs
#     start_idx = np.searchsorted(sorted_char_char_player_ids, player_id, side='left')
#     end_idx = np.searchsorted(sorted_char_char_player_ids, player_id, side='right')
    
#     # Map the sorted indices back to the original array indices
#     player_indices = sorted_indices[start_idx:end_idx]
    
#     return player_indices




In [190]:
# Sort column names and cache them for faster lookup
alt2_char_char_columns = char_vs_char_player_rankings_weekly_alt2_df.columns
alt2_sorted_indices = np.argsort(alt2_char_char_columns)
alt2_sorted_columns = alt2_char_char_columns[alt2_sorted_indices]
alt2_sorted_columns_id = [label.split('/')[0] for label in alt2_char_char_columns[alt2_sorted_indices]]
alt2_sorted_columns_id_char = [label.split('/')[0] + '/' + label.split('/')[1] for label in alt2_char_char_columns[alt2_sorted_indices]]

# Create a mapping for easy access
sorted_alt2_column_mapping = {
    "sorted_indices": alt2_sorted_indices,
    "sorted_columns": alt2_sorted_columns,
    "sorted_columns_id": alt2_sorted_columns_id,
    "sorted_columns_id_char": alt2_sorted_columns_id_char
}

def get_alt2_player_columns(player_id, df=char_vs_char_player_rankings_weekly_alt2_df,sorted_column_mapping=sorted_alt2_column_mapping):
    sorted_indices = sorted_column_mapping["sorted_indices"]
    sorted_columns = sorted_column_mapping["sorted_columns_id"]
    
    # Find the start and end range for player_id
    start_idx = np.searchsorted(sorted_columns, player_id, side='left')
    end_idx = np.searchsorted(sorted_columns, player_id, side='right')
    
    # If no columns are found, return an empty DataFrame
    if start_idx == end_idx:
        return pd.DataFrame()  # or handle the case in another way if preferred
    
    # Map sorted indices back to original column order and subset DataFrame
    player_indices = sorted_indices[start_idx:end_idx]
    return df.iloc[:, player_indices]

def get_alt2_player_character_columns(player_id, player_char, df=char_vs_char_player_rankings_weekly_alt2_df, sorted_column_mapping=sorted_alt2_column_mapping):
    sorted_indices = sorted_column_mapping["sorted_indices"]
    sorted_columns = sorted_column_mapping["sorted_columns_id_char"]
    
    # Format search string as player_id/player_character/opponent_character
    search_pattern = player_id + '/' + player_char
    
    # Find the start and end range for player columns
    start_idx = np.searchsorted(sorted_columns, search_pattern, side='left')
    end_idx = np.searchsorted(sorted_columns, search_pattern, side='right')
    
    if start_idx == end_idx:
        return pd.DataFrame()  # or handle the case in another way if preferred
    
    # Map sorted indices back to original column order and subset DataFrame
    relevant_columns = sorted_indices[start_idx:end_idx]
    return df.iloc[:, relevant_columns]

def get_alt2_column(player_id, player_char, opponent_char, df=char_vs_char_player_rankings_weekly_alt2_df, sorted_column_mapping=sorted_alt2_column_mapping):
    sorted_indices = sorted_column_mapping["sorted_indices"]
    sorted_columns = sorted_column_mapping["sorted_columns"]
    
    # Format search string as player_id/player_character/opponent_character
    search_pattern = player_id + '/' + player_char + '/' + opponent_char
    
    # Find the start and end range for player columns
    idx = np.searchsorted(sorted_columns, search_pattern, side='left')

    return df.iloc[:, sorted_indices[idx]]


In [195]:
# Sort column names and cache them for faster lookup
alt2_rds_char_char_columns = char_vs_char_player_rankings_weekly_alt2_rds_df.columns
alt2_rds_sorted_indices = np.argsort(alt2_rds_char_char_columns)
alt2_rds_sorted_columns = alt2_rds_char_char_columns[alt2_rds_sorted_indices]
alt2_rds_sorted_columns_id = [label.split('/')[0] for label in alt2_rds_char_char_columns[alt2_rds_sorted_indices]]
alt2_rds_sorted_columns_id_char = [label.split('/')[0] + '/' + label.split('/')[1] for label in alt2_rds_char_char_columns[alt2_rds_sorted_indices]]

# Create a mapping for easy access
sorted_alt2_rds_column_mapping = {
    "sorted_indices": alt2_rds_sorted_indices,
    "sorted_columns": alt2_rds_sorted_columns,
    "sorted_columns_id": alt2_rds_sorted_columns_id,
    "sorted_columns_id_char": alt2_rds_sorted_columns_id_char
}

def get_alt2_rds_player_columns(player_id, df=char_vs_char_player_rankings_weekly_alt2_rds_df,sorted_column_mapping=sorted_alt2_rds_column_mapping):
    sorted_indices = sorted_column_mapping["sorted_indices"]
    sorted_columns = sorted_column_mapping["sorted_columns_id"]
    
    # Find the start and end range for player_id
    start_idx = np.searchsorted(sorted_columns, player_id, side='left')
    end_idx = np.searchsorted(sorted_columns, player_id, side='right')
    
    # If no columns are found, return an empty DataFrame
    if start_idx == end_idx:
        return pd.DataFrame()  # or handle the case in another way if preferred
    
    # Map sorted indices back to original column order and subset DataFrame
    player_indices = sorted_indices[start_idx:end_idx]
    return df.iloc[:, player_indices]

def get_alt2_rds_player_character_columns(player_id, player_char, df=char_vs_char_player_rankings_weekly_alt2_rds_df, sorted_column_mapping=sorted_alt2_rds_column_mapping):
    sorted_indices = sorted_column_mapping["sorted_indices"]
    sorted_columns = sorted_column_mapping["sorted_columns_id_char"]
    
    # Format search string as player_id/player_character/opponent_character
    search_pattern = player_id + '/' + player_char
    
    # Find the start and end range for player columns
    start_idx = np.searchsorted(sorted_columns, search_pattern, side='left')
    end_idx = np.searchsorted(sorted_columns, search_pattern, side='right')
    
    if start_idx == end_idx:
        return pd.DataFrame()  # or handle the case in another way if preferred
    
    # Map sorted indices back to original column order and subset DataFrame
    relevant_columns = sorted_indices[start_idx:end_idx]
    return df.iloc[:, relevant_columns]

def get_alt2_rds_column(player_id, player_char, opponent_char, df=char_vs_char_player_rankings_weekly_alt2_rds_df, sorted_column_mapping=sorted_alt2_rds_column_mapping):
    sorted_indices = sorted_column_mapping["sorted_indices"]
    sorted_columns = sorted_column_mapping["sorted_columns"]
    
    # Format search string as player_id/player_character/opponent_character
    search_pattern = player_id + '/' + player_char + '/' + opponent_char
    
    # Find the start and end range for player columns
    idx = np.searchsorted(sorted_columns, search_pattern, side='left')

    return df.iloc[:, sorted_indices[idx]]
